In [ ]:
# Package imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
data = pd.read_csv("data/data.csv")
print("data type:",type(data))
print("data shape:", data.shape)
print("data:")
print(data.head().to_string(index=False))

# Split the data into train and test
train_dataset = data.sample(frac=0.8, random_state=0)
test_dataset = data.drop(train_dataset.index)

# Normalize (divide by max value)
x_train = train_dataset.multiply(1/100).iloc[:,0:6]
y_train = train_dataset.multiply(1/100).iloc[:,6:12]
x_test = test_dataset.multiply(1/100).iloc[:,0:6]
y_test = test_dataset.multiply(1/100).iloc[:,6:12]
print('Train shape',x_train.shape)
print('Test shape',x_test.shape)

In [ ]:
# Define Sequential Model
model = keras.Sequential(
    [   keras.Input(shape=(6,)),
        keras.layers.Dense(48, activation='relu'),
        keras.layers.Dense(96, activation='relu'),
        keras.layers.Dense(192, activation='relu'),
        keras.layers.Dense(96, activation='relu'),
        keras.layers.Dense(48, activation='relu'),
        keras.layers.Dense(6)
    ]
)
model.summary()

In [ ]:
# loss, optimizer and metrics
model.compile(loss=keras.losses.MeanSquaredError(), 
              optimizer=keras.optimizers.Adam(learning_rate=0.001), 
              metrics=['mae'])

In [ ]:
class haltCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss') <= 1.0e-06):
            print("\n\n\nReached loss limit, stopping training!\n\n")
            self.model.stop_training = True
trainingStopCallback = haltCallback()

In [ ]:
# training
model.fit(x_train, y_train, epochs=100,batch_size=64, verbose=1, callbacks=[trainingStopCallback])

In [ ]:
pred = model.predict(x_train, verbose=0)
test_result = np.round(pred*100).astype(int) - np.round(y_train*100).astype(int)
test_result_row_sums = test_result.abs().sum(axis=1)
num_correct=len(test_result_row_sums[test_result_row_sums == 0])
print('Performance (train set):', (100*num_correct)/test_result.shape[0], '%')

pred = model.predict(x_test, verbose=0)
y_round = np.round(y_test*100).astype(int)
test_result = np.round(pred*100).astype(int) - np.round(y_test*100).astype(int)
test_result_row_sums = test_result.abs().sum(axis=1)
num_correct= len(test_result_row_sums[test_result_row_sums == 0])
print('Performance (test set):', (100*num_correct)/test_result.shape[0], '%')


In [ ]:
# Check prediction of first test input
x1 = x_test.iloc[0:1,:]
y1 = y_test.iloc[0:1,:]
x1_pred = model.predict(x1, verbose=0)
print('x1',(x1*100).astype(int).to_string(index=False))
print('y1',(y1*100).astype(int).to_string(index=False))
print('prediction: \n',np.round(x1_pred*100).astype(int)[0,:])



In [ ]:
#model.save("mathpyramid")

In [ ]:
#model = keras.models.load_model("mathpyramid_vectors_99.2_correct")
#model.summary()

In [ ]:
# Predict with higher values (<= 1000)
data = {'x1': [175],
        'x2': [114],
        'x3': [102],
        'x4': [175+114],
        'x5': [114 + 102],
        'x6': [175+114 + 114 + 102]
        }  
# Create the pandas DataFrame
df = pd.DataFrame(data)
print(df)
pred = model.predict(df/1000)
print(np.round(pred*1000).astype(int))